In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
def comma_collapse(x):
    """Utility function to remove non-strings before apply str.join()
    
    :param x: list of items to apply str.join() to
    :return: len 1 str of joined items in x
             Returns empty string if no strings in x
    """
    x_str = [k for k in x if isinstance(k, str)]

    if x is None:
        comma_sep_list = ""
    else:
        comma_sep_list = ", ".join(x_str)
    return comma_sep_list


def rename_reverse_colon(df):
    """Some series of questions take opposite form of most questions that use colons
    This is a helper to rename them.
    
    Examples: 
        * "use a debugger:When developing in Python, how often do you …?"
        * "Computer graphics:To what extent are you involved in the following activities?"
    
    Most questions with a `:` in them take a different form where the identifying
    info is after the colon. This set of columns requires different processing.
    
    This function should be applied before other processing that relies on `:`.
    
    :param df: jetbrains python survery dataframe
    :return: df with columns of interest renamed
    """
    phrases = [
        {
            "key_phrase": "When developing in Python, ",
            "new_form": "how often do you {x}?",
        },
        {
            "key_phrase": "To what extent are you involved",
            "new_form": "how involved are you with {x}?",
        },
    ]

    name_map = {}
    for p in phrases:
        phrase_df = df.filter(like=p["key_phrase"])
        phrase_cols = phrase_df.columns

        for col in phrase_cols:
            x = col.split(":")[0]
            q = p["new_form"].format(x=x)
            name_map[col] = q

    return df.rename(columns=name_map)

<IPython.core.display.Javascript object>

## Data comments:

* All questions were multiple choice. The only columns with more than 24 unique responses are the id field and the country field.
* All fields except the id are strings
* There is a response category for "Other – Write In:" in many cols. We don't have the write in responses in our current data.

----

## Data fields/groups of interest
(after processing) If not listed here, column can go through default categorical encoding
    
### Do you consider yourself as a Data-Scientist?'

* Responses: `['Other – Write In:', 'No', 'Yes']`
* Binary and treat write-ins as nan? One-hot?

In [4]:
data_path = "data/python_psf_external_19.csv"

# Was getting warning about dtypes, took lazy way out with low_memory
# instead of setting dtype arg
raw_df = pd.read_csv(data_path, low_memory=False)
raw_df = rename_reverse_colon(raw_df)
raw_df.shape

(47308, 290)

<IPython.core.display.Javascript object>

In [5]:
colon_cols = [c for c in raw_df if ":" in c]
non_colon_cols = [c for c in raw_df if c not in colon_cols]

colon_col_names = pd.Series(colon_cols)
colon_unique_qs = colon_col_names.str.split(":").str[-1].unique()

collapsed_series = []
for qoi in tqdm(colon_unique_qs):
    df_oi = raw_df.filter(like=qoi)
    collapsed = df_oi.apply(comma_collapse, axis=1)
    collapsed.name = qoi.strip()
    collapsed_series.append(collapsed)


colon_df = pd.concat(collapsed_series, axis=1)
df = pd.concat((raw_df[non_colon_cols], colon_df), axis=1)
df = df.set_index("response_id")

# Removing "Not sure" responses to "How many people work for your company / organization?"
# Only drops ~3% of records, and not sure how to map them ordinally with rest of
# responses
df = df[df["How many people work for your company / organization?"] != "Not sure"]

df.head()

,Is Python the main language you use for your current projects?,How long have you been programming in Python?,How many years of professional coding experience do you have?,For what purposes do you mainly use Python?,how involved are you with Computer graphics?,how involved are you with Data analysis?,how involved are you with Desktop development?,how involved are you with DevOps / System administration / Writing automation scripts?,how involved are you with Educational purposes?,how involved are you with Embedded development?,...,How do you develop for the cloud?,What operating system(s) are your development environment?,"Which Python unit-testing framework(s) do you use, if any?","What ORM(s) do you use together with Python, if any?","Which database(s) do you regularly use, if any?","Which of the following Big Data tool(s) do you use, if any?",Which Continuous Integration (CI) system(s) do you regularly use?,"Which configuration management tools do you use, if any?",What editors/IDEs do you use for Python development in addition to your primary IDE?,Which of the following best describes your job role(s)?
response_id,,,,,,,,,,,,,,,,,,,,,
10,Yes,6–10 years,11+ years,Both for work and personal,NaN,NaN,NaN,secondary activity,NaN,NaN,...,Locally with virtualenv (or similar),macOS,"mock, pytest, unittest",Django ORM,"MySQL, Oracle Database",Apache Kafka,"AppVeyor, Gitlab CI, Jenkins / Hudson, Travis CI","Chef, Puppet, Salt",Vim,Developer / Programmer
100,Yes,3–5 years,1–2 years,For work,NaN,NaN,NaN,NaN,NaN,NaN,...,"In Docker containers, In virtual machines",macOS,"mock, pytest, unittest",Django ORM,PostgreSQL,None,Gitlab CI,None,PyCharm Professional Edition,Developer / Programmer
1000,Yes,3–5 years,3–5 years,Both for work and personal,NaN,secondary activity,NaN,NaN,NaN,NaN,...,With local system interpreter,"Linux, macOS",pytest,None,None,Apache Kafka,None,None,"PyCharm Community Edition, Sublime Text",Other – Write In:
10000,Yes,1–2 years,Less than 1 year,"For personal, educational or side projects",NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,


<IPython.core.display.Javascript object>

### Mapping Ordinal Categories to Integers

In [6]:
# Convert df to nums
ordinal_mappers = {
    "Is Python the main language you use for your current projects?": {
        "Yes": 3,
        "No, I use Python as a secondary language": 2,
        "No, I don’t use Python for my current projects": 1,
    },
    "How long have you been programming in Python?": {
        "11+ years": 5,
        "6–10 years": 4,
        "3–5 years": 3,
        "1–2 years": 2,
        "Less than 1 year": 1,
    },
    "How many years of professional coding experience do you have?": {
        "11+ years": 5,
        "6–10 years": 4,
        "3–5 years": 3,
        "1–2 years": 2,
        "Less than 1 year": 1,
    },
    "How often do you use your primary IDE?": {
        "Daily": 4,
        "Weekly": 3,
        "Monthly": 2,
        "Less frequently": 1,
    },
    "How many people are in your project team?": {
        "More than 40 people": 5,
        "21-40 people": 4,
        "13-20 people": 3,
        "8-12 people": 2,
        "2-7 people": 1,
    },
    "How many people work for your company / organization?": {
        "More than 5,000": 7,
        "1,001–5,000": 6,
        "501–1,000": 5,
        "51–500": 4,
        "11–50": 3,
        "2–10": 2,
        "Just me": 1,
        "Not sure": -1,
    },
    "Could you tell us your age range?": {
        "60 or older": 6,
        "50–59": 5,
        "40–49": 4,
        "30–39": 3,
        "21–29": 2,
        "18–20": 1,
    },
}


for col_name, ordinal_map in ordinal_mappers.items():
    df[col_name] = df[col_name].replace(ordinal_map)

<IPython.core.display.Javascript object>

In [7]:
cols_oi = df.filter(like="how often do you").columns
ordinal_map = {"Often": 3, "From time<br />to time": 2, "Never or<br />Almost never": 1}

df[cols_oi] = df[cols_oi].replace(ordinal_map)

<IPython.core.display.Javascript object>

In [8]:
cols_oi = df.filter(like="how involved").columns
ordinal_map = {"primary activity": 3, "secondary activity": 2, "hobby": 1}

df[cols_oi] = df[cols_oi].replace(ordinal_map)

<IPython.core.display.Javascript object>

###  Mapping Comma Sep List (csl) Cols to Dummies

In [9]:
csl_cols = [
    "What other language(s) do you use?",
    "What do you use Python for?",
    "What do you typically use to upgrade your Python version?",
    "Do you use any of the following tools to isolate Python environments, if any?",
    "What web frameworks / libraries do you use in addition to Python?",
    "What data science framework(s) do you use in addition to Python?",
    "Which of the following frameworks / libraries do you use in addition to Python?",
    "Which of the following cloud platforms do you use?",
    "How do you run code in the cloud (in the production environment)?",
    "How do you develop for the cloud?",
    "What operating system(s) are your development environment?",
    "Which Python unit-testing framework(s) do you use, if any?",
    "What ORM(s) do you use together with Python, if any?",
    "Which database(s) do you regularly use, if any?",
    "Which of the following Big Data tool(s) do you use, if any?",
    "Which Continuous Integration (CI) system(s) do you regularly use?",
    "Which configuration management tools do you use, if any?",
    "What editors/IDEs do you use for Python development in addition to your primary IDE?",
    "Which of the following best describes your job role(s)?",
]

non_csl_cols = [c for c in df if c not in csl_cols]
non_csl_df = df[non_csl_cols]

one_hot_dfs = []
for col in tqdm(csl_cols):
    one_hot_df = df[col].str.get_dummies(", ")
    one_hot_dfs.append(one_hot_df)

full_one_hot_df = pd.concat(one_hot_dfs, axis=1)
df = pd.concat((df, full_one_hot_df), axis=1)

<IPython.core.display.Javascript object>

###  Mapping Python Version to Float

In [10]:
py_version_cols = [
    "Which version of Python do you use the most?",
    "Which version of Python 2 do you use the most?",
    "Which version of Python 3 do you use the most?",
]

# Removing the 2 people who gave minor versions for the 2/3 question... weirdos
df = df[df[py_version_cols].isna().sum(axis=1) > 0]

for col in py_version_cols:
    df[col] = df[col].str.replace(r"Python (.+)", r"\1")
    df[col] = df[col].astype(float)

df["python_version"] = df[py_version_cols].max(axis=1)
df = df.drop(columns=py_version_cols)

<IPython.core.display.Javascript object>